In [2]:
import pandas as pd
import re
from eunjeon import Mecab
import collections

In [3]:
#dirr = 'C:/Users/rsh15/Google Drive/crawler_data/estate_news/'
dirr = 'C:/Users/shic/Desktop/shinhan_simon/google_drive/crawler_data/estate_news/'

In [4]:
news_df = pd.read_csv(dirr+'RAA_부동산114뉴스기사_유승훈.csv')
news_df = news_df.dropna()

In [5]:
news_df['yearmonth'] = news_df['NEWS_WRT_YMD'].str.slice(stop=7).str.replace('.','',regex=False)
news_df

,NEWS_TITLE_NM,NEWS_WRT_YMD,NEWS_MEDIA_NM,NEWS_BODY_CNTT,yearmonth
0,"[주간부동산] 매매·전세, 연말연시 맞아 `한산`",2012.01.01,이데일리,"- 매매, 서울 0.05%↓, 신도시 0.02%↓, 수도권 0.02%↓- 전세, 서...",201201
1,"연말 거래 한산, 매매도 전세도 약세 지속",2012.01.01,조선비즈,서울 아파트 값이 2주 연속 하락세다. 2011년 마지막 주 시장은 비수기로 한산했...,201201
2,"2012년 아파트 공급 줄고, 오피스텔 늘고",2012.01.01,헤럴드POP,"새해에는 브랜드 아파트 공급은 작년보다 줄고, 대표적인 수익형부동산 상품인 오피스텔...",201201
3,수도권 집 한 채에 지방 아파트 몇 채 산다고?,2012.01.01,헤럴드POP,수도권 아파트 한 채 값에 지방 아파트 여러 채를 산다는 말도 옛말이 됐다. 지방 ...,201201
4,고급빌라 맞수 청담동 vs 한남동…“유엔빌리지(한남동) 살아요” ‘70억 펜트하우스...,2012.01.01,매경이코노미,한남동 유엔빌리지. / 청담동 마크힐스. TV 드라마 속 부잣집에서 전화를 받는 사...,201201
...,...,...,...,...,...
45907,[부동산캘린더] 3기 신도시 사전청약 1순위 접수…분양시장 후끈,2021.07.31,연합뉴스,(서울=연합뉴스) 김동규 기자 = 다음 주에는 3기 신도시 사전청약 1순위 접수를 ...,202107
45908,[분양캘린더]8월 첫째 주 1만4368가구 분양…견본주택 3곳 개관,2021.07.31,뉴시스,전국 20개 단지…일반분양 1만86가구[서울=뉴시스] 31일 부동산114에 따르면 ...,202107
45909,올 상반기 청약률 상위 단지는 초등학교 도보 통학권,2021.07.31,데일리안,"분양 시장 주 수요층 3040세대, 자녀 교육 우선안전한 통학 환경 선호해올해 상반...",202107
45910,집값에 치이고 전셋값에 또 치이고,2021.07.31,비즈니스워치,[집값 톡톡]전셋값→집값 상승 악순환서울·수도권·지방까지 전셋값 상승 확산집값은 '...,202107


In [6]:
dates = news_df['yearmonth'].unique().tolist()
dates[0:10]

['201201',
 '201202',
 '201203',
 '201204',
 '201205',
 '201206',
 '201207',
 '201208',
 '201209',
 '201210']

In [7]:
def news_article_preprocess(article_cnt):
    content_txt = re.sub(r'[\(\[].*?[\)\]]','',article_cnt)
    content_txt = re.sub('<.*?>','',content_txt)
    content_txt = re.sub('＜.*?＞','',content_txt)
    content_txt = re.sub('\s{2,}','',content_txt)
    content_txt = content_txt.replace('부동산114','')
    return content_txt

In [13]:
news_df['NEWS_BODY_CNTT'] = news_df['NEWS_BODY_CNTT'].map(news_article_preprocess)
news_df['NEWS_BODY_CNTT'][23123]

"평택 소사벌 푸르지오 투시도© News1대우건설·대림산업 컨소시엄, 택지지구내 메이저 브랜드전가구 중소형 4베이 설계에 혁신기술 도입 진희정 기자 = 다음달 개통하는 수서발고속철도 개통과 내년 4월 가동하는 고덕삼성산업단지에 대한 기대감으로 평택이 들썩이고 있다. SRT 개통 자체가 평택시 전체 인구를 끌어올리는 촉매가 될 것이라는 의미에서다. 또 산업단지와 지역 간 연계성과 성장 가능성을 고려하면 평택의 미래가치는 더 높아질 것이란 얘기다.각종 개발계획으로 주목받고 있는 평택 소사벌 택지개발지구에서 대우건설과 대림산업이 손잡고 '평택 소사벌 푸르지오'를 분양한다. 25일 모델하우스를 오픈한 평택 소사벌 푸르지오는 지하 1층~지상 25층 아파트 6개동 규모로 전용면적 Δ83㎡A 334가구 Δ83㎡A1 42가구 Δ83㎡B 78가구 Δ83㎡B1 21가구 Δ83㎡C 91가구 등 총 566가구로 전 세대가 수요층이 많은 중소형 규모로 구성돼 있다. 분양가는 3.3㎡ 당 평균 900만원 중반대로 예상되며 계약금정액제, 중도금 전액 무이자가 적용된다. ◇연이은 대형호재로 주목…아파트 값도 오르고 있어삼성 및 LG전자 등 대기업 산업단지 조성사업이 속도를 내고 있는 데다 서울권 접근성 향상에 기여할 SRT 지제역도 다음달 개통한다. 여기에 고덕산업단지 내에 삼성전자가 100조원 이상을 투자해 조성하는 아시아 최대 규모의 차세대 반도체 공장의 1기 라인이 내년 상반기에 가동을 앞두고 있다. LG전자가 조성 중인 99만여㎡ 규모의 평택 진위2산업단지도 내년 준공을 목표로 하고 있다. 실제 최근 5년간 평택시 집값은 상승세를 기록하고 있다.  자료를 보면 평택시의 지난달 3.3㎡당 아파트값은 평균 704만원으로 지난 2012년 10월 대비 14.66%의 상승률을 기록했다. 같은 기간 경기도 전체 3.3㎡당 아파트 평균 매매가 상승률인 10.22%를 웃도는 수치다. 최근 3년간 1~10월까지 평택시 아파트 거래량도 꾸준히 증가하고 있다. 국토교통부 통계누리 자료에 따르면 201

In [ ]:
l
for dd in dates:
    tmp_df = news_df[news_df['yearmonth']==dd]
    
    

In [14]:
news_df

,NEWS_TITLE_NM,NEWS_WRT_YMD,NEWS_MEDIA_NM,NEWS_BODY_CNTT,yearmonth
0,"[주간부동산] 매매·전세, 연말연시 맞아 `한산`",2012.01.01,이데일리,"- 매매, 서울 0.05%↓, 신도시 0.02%↓, 수도권 0.02%↓- 전세, 서...",201201
1,"연말 거래 한산, 매매도 전세도 약세 지속",2012.01.01,조선비즈,서울 아파트 값이 2주 연속 하락세다. 2011년 마지막 주 시장은 비수기로 한산했...,201201
2,"2012년 아파트 공급 줄고, 오피스텔 늘고",2012.01.01,헤럴드POP,"새해에는 브랜드 아파트 공급은 작년보다 줄고, 대표적인 수익형부동산 상품인 오피스텔...",201201
3,수도권 집 한 채에 지방 아파트 몇 채 산다고?,2012.01.01,헤럴드POP,수도권 아파트 한 채 값에 지방 아파트 여러 채를 산다는 말도 옛말이 됐다. 지방 ...,201201
4,고급빌라 맞수 청담동 vs 한남동…“유엔빌리지(한남동) 살아요” ‘70억 펜트하우스...,2012.01.01,매경이코노미,한남동 유엔빌리지. / 청담동 마크힐스. TV 드라마 속 부잣집에서 전화를 받는 사...,201201
...,...,...,...,...,...
45907,[부동산캘린더] 3기 신도시 사전청약 1순위 접수…분양시장 후끈,2021.07.31,연합뉴스,김동규 기자 = 다음 주에는 3기 신도시 사전청약 1순위 접수를 비롯해 전국에서 ...,202107
45908,[분양캘린더]8월 첫째 주 1만4368가구 분양…견본주택 3곳 개관,2021.07.31,뉴시스,전국 20개 단지…일반분양 1만86가구 31일 에 따르면 8월 첫째 주에는 전국 2...,202107
45909,올 상반기 청약률 상위 단지는 초등학교 도보 통학권,2021.07.31,데일리안,"분양 시장 주 수요층 3040세대, 자녀 교육 우선안전한 통학 환경 선호해올해 상반...",202107
45910,집값에 치이고 전셋값에 또 치이고,2021.07.31,비즈니스워치,전셋값→집값 상승 악순환서울·수도권·지방까지 전셋값 상승 확산집값은 '고점 예고' ...,202107


In [66]:
df_content = pd.concat(list_tmp).reset_index(drop=True)
df_content

,dates,content
0,201201,"- 매매, 서울 0.05%↓, 신도시 0.02%↓, 수도권 0.02%↓- 전세, 서..."
1,201202,투자분석 보고서분담금 9000만원까지 절감1만 가구가 넘는 대단지로 변모하는 서울...
2,201203,"동영상 뉴스【 앵커멘트 】 김포 한강과 파주 운정, 그리고 인천 송도·청라 지구...."
3,201204,국내 최대 포털 네이버가 오픈마켓 서비스 ‘샵N’의 문을 열고 업계로부터 뭇매를 ...
4,201205,2008년 금융위기 이후 가격이 크게 내린 아파트 중에서 소액 투자가 가능한 중소형...
...,...,...
110,202103,"잠실 엘스, 2억 내린 전세매물 등장두달새 25% 증가…서울·경기 비슷“다만 입주 ..."
111,202104,"- 대전 하늘채 스카이앤 2차, 5월중 분양일정 순항… 상담 가능한 ‘사전홍보관’ ..."
112,202105,동영상 뉴스서울 아파트값이 재건축 단지를 중심으로 다시 들썩이는 조짐이 나타나자 오...
113,202106,매년 6월 1일 기준 보유세 부과6월부터 매물잠김 시작되는 이유올해는 양도세 중과까...


In [67]:
tagger = Mecab()
tagger_list = []
for roww in df_content.iterrows():
    tagged = tagger.nouns(roww[1]['content'])
    tagged = [s for s in tagged if len(s) > 1]
    tagged_counter = collections.Counter(tagged)
    tagged_common  = tagged_counter.most_common(100)
    
    word_list = []
    count_list = []
    for word,count in tagged_common:
        word_list.append(word)
        count_list.append(count)
    
    tagger_list.append(pd.DataFrame({
        'date'  : roww[1]['dates'],
        'word'  : word_list,
        'count' : count_list
    }))

In [68]:
tagger_100[tagger_100['date']=='201204']

,date,word,count
0,201204,성폭행,1
1,201204,가담,1
2,201204,아이돌,1
3,201204,멤버,1
4,201204,형사,1
5,201204,처벌,1
6,201204,검토,1
7,201204,이하이,1
8,201204,박지민,1
9,201204,백아연,1


In [12]:
tagger_100 = pd.concat(tagger_list)
tagger_100.to_csv(dirr+'wordcount_df_100.csv',index=False,encoding='UTF-8-sig')

In [118]:
tagger = Mecab()
tagger_list = []
for roww in df_content.iterrows():
    tagged = tagger.nouns(roww[1]['content'])
    tagged = [s for s in tagged if len(s) > 1]
    tagged_counter = collections.Counter(tagged)
    tagged_common  = tagged_counter.most_common(50)
    
    word_list = []
    count_list = []
    for word,count in tagged_common:
        word_list.append(word)
        count_list.append(count)
    
    tagger_list.append(pd.DataFrame({
        'date'  : roww[1]['dates'],
        'word'  : word_list,
        'count' : count_list
    }))

In [119]:
tagger_50 = pd.concat(tagger_list)
tagger_50.to_csv(dirr+'wordcount_df_50.csv',index=False,encoding='UTF-8-sig')